TRANSFER FUNCTION 

Let's start with adding necessary libraries !


Do NOT forget to run codes in order !

In [0]:
!pip install control
import control as cnt
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import math


Transfer functions specify the relationship between input and output.

$G(s) = \frac{b_ms^m+b_{m-1}s^{m-1}+...+b_{1}s+b_0}{a_ns^n+a_{n-1}s^{n-1}+...+a_{1}s+a_0} =\frac{N(s)}{D(s)}$

To create transfer function in laplace form :

Numerator and denominator coefficients should be given in a matrix form in descending order then cnt.tf(num,den) or cnt.TransferFunction(num,den) commands are used.




In [0]:
num = [2, 3, 1]
den = [5, 8, 9, 45]
TF = cnt.TransferFunction(num,den) 
print("G(s)", TF, sep = "=")

If denominator's order is higher than numerator: ($n\geq m$)

 the roots of D(s) = 0 which is also named as characteristic equation give us the poles of the system. Poles can be determined by cnt.pole() command.

 the roots of N(s) = 0 give us zeros of the system. Poles can be determined by cnt.zero() command.
 

In [0]:
Poles = cnt.pole(TF)
Zeros = cnt.zero(TF)
print("Poles of the system=", Poles)
print("Zeros of the system=", Zeros)

Important Notes up to here:
  - Transfer function is a property of a system and is independent of input.
  - Transfer functions of different physical systems can be the same. 



---



---


Example 1


---



---






<figure>
<center>
<img src='https://www.howacarworks.com/illustration/127/coil-spring.png' />
<figcaption>Figure 1</figcaption></center>



The suspension system of a vehicle is shown in *Figure 1* and a simple dynamic model of a vehicle travelling on a rough road surface is shown in *Figure 2*. The mass represents the mass of the vehicle body. The spring and damper represent the suspension springs and dampers.

<figure>
<center>
<img src='https://drive.google.com/uc?id=1V873QQ7QDDtmHT05wnFRIb1wqKOSAOXI' />
<figcaption>Figure 2</figcaption></center>
</figure>

The input is the road profile displacement, ie. $z $. If the output is the displacement y of the body. 


*   Obtain input-output relationship as $m\ddot{y}+c\dot{y}+ky=c\dot{z}+kz$ .
*   In order to get transfer function, we should get the laplace transform of input-output expression.
$ms^2Y(s)+csY(s)+kY(s)=csZ(s)+kZ(s)$
then 
$\dfrac {output} {input} = \dfrac {Y(s)} {Z(s)} = \dfrac {cs+k} {ms^2+cs+k}  $.



Let's specify the m, c, k values.

In [0]:
m = 1600     #[kg]
c = 6000     #[N/m/s]
k = 784000   #[N/m]
num = [c,k]
den = [m,c,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)
T,y = cnt.step_response(TF,t)   #step response of the system from transfer function
plt.plot(T,y)
plt.xlabel("Time [s]")
plt.ylabel("Displacement of car body in y-axis")
plt.show()

Now, it's your turn. You can change the m, c, k values and see the difference in transfer function, displacement and oscillations.

In [0]:
m =      #[kg]
c =      #[N/m/s]
k =      #[N/m]
num = [c,k]
den = [m,c,k]
TF = cnt.tf(num,den)
print("G(s)=",TF)
#Displacement vs. Time Plot
t = np.linspace(0,5,501)
T,y = cnt.step_response(TF,t)   #step response of the system from transfer function
plt.plot(T,y)
plt.xlabel("Time [s]")
plt.ylabel("displacement of car body in y-axis")
plt.show()